<a href="https://colab.research.google.com/github/ansem7/cs199specialproject/blob/main/TorchAttacksTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Install torchattacks**

In [282]:
!pip install torchattacks
# ! wget -O apple.jpg https://images.everydayhealth.com/images/diet-nutrition/apples-101-about-1440x810.jpg # Replace this(the second) url for another image. (It's currently an apple)
! wget -O imagenet_class_index.json https://github.com/ansem7/cs199specialproject/blob/e22b91ccf47246a081323abbf952b0d030b6faf6/imagenet_class_index.json

--2024-06-08 12:34:38--  https://raw.githubusercontent.com/raghakot/keras-vis/master/resources/imagenet_class_index.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35363 (35K) [text/plain]
Saving to: ‘imagenet_class_index.json’

imagenet_class_inde 100%[===================>]  34.53K  --.-KB/s    in 0.01s   

2024-06-08 12:34:39 (3.30 MB/s) - ‘imagenet_class_index.json’ saved [35363/35363]



## **2. Import the necessary libraries:**

In [283]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, f1_score
import os
import time

## **3. Load the pretrained ResNet50 model:**

In [284]:
model = resnet50(pretrained=True).eval()

## **4. Define the transformation for your image:**

In [285]:
# Define the transformation
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),  # Convert image to RGB if it's not
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## **5. Load your images**

In [286]:
from google.colab import drive
import os

# Define the directory path
dir_path = "/content/test"

# Create the directory
os.makedirs(dir_path, exist_ok=True)

drive.mount('/content/drive')

# Path to your images in Google Drive
image_path = '/content/drive/My Drive/partial_images2'

# image_path = '/content/test'

# Load images from the directory
image_folder = datasets.ImageFolder(image_path, transform=transform)

# Create a data loader
image_loader = torch.utils.data.DataLoader(image_folder, batch_size=32)

# image = Image.open('apple.jpg')
# image = transform(image).unsqueeze(0)  <--- these two are used for a single image

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **6. Create an instance of the chosen attack**

In [287]:
from torchattacks import FGSM, PGD, CW, PGDL2, NIFGSM, Pixle, SINIFGSM, VMIFGSM, VNIFGSM, LGV

# Define the attack
# FGSM
#attack = FGSM(model, eps=0.3)

# PGD
#attack = PGD(model, eps=8/255, alpha=2/255, steps=2)

# CW
#attack = CW(model, c=1, kappa=0, steps=2, lr=0.01)

# PGDL2
#attack = PGDL2(model, eps=0.3, alpha=0.01, steps=2)

# NIFGSM
#attack = NIFGSM(model, eps=0.2)

# Pixle
attack = Pixle(model, x_dimensions=(2, 10), y_dimensions=(2, 10), pixel_mapping='random', restarts=20, max_iterations=10, update_each_iteration=False)

# SINIFGSM
#attack = SINIFGSM(model, eps=8/255, alpha=2/255, steps=2, decay=1.0, m=5)

# VMIFGSM
#attack = VMIFGSM(model, eps=8/255, alpha=2/255, steps=2, decay=1.0)

# VNIFGSM
#attack = VNIFGSM(model, eps=8/255, alpha=2/255, steps=2, decay=1.0, N=5, beta=3/2)

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []


## **7. Apply the attack to your image:**

In [288]:
# adv_image = attack(image, torch.tensor([322]))  # 948 is the ImageNet class index for 'apple'

import json
import numpy as np

# Load the ImageNet class index file
with open('imagenet_class_index.json') as f:
    class_idx = json.load(f)

# Now you can use this dictionary to get the label name from an index
#def get_label_name(index):

#    return class_idx[index]

def get_label_name(index):
      return class_idx[f"{index}"]

predicted_labels = []
true_labels = []

total_attack_time = 0  # Initialize a variable to store the total attack time

# Iterate over the images
for i, (images, labels) in enumerate(image_loader):

    #print(f"{i}: {labels}")

    # Start the timer before the attack
    start_time = time.time()

    # Apply the attack to the images
    adv_images = attack(images, labels)

    # End the timer after the attack and add the elapsed time to the total
    total_attack_time += time.time() - start_time

    # Pass the adversarial images through the model
    outputs = model(adv_images)

    # Get the predicted labels
    _, preds = torch.max(outputs, 1)

    # Convert the predicted labels to their classification names
    predicted_labels_names = [get_label_name(label)[1] for label in preds.tolist()]

    # Convert the true labels to their classification names
    true_labels_names = [image_loader.dataset.classes[label] for label in labels.tolist()]

    predicted_labels.extend(predicted_labels_names)
    true_labels.extend(true_labels_names)

    #print(f"Predicted labels names: {predicted_labels_names}")
    #print(f"Actual labels: {true_labels_names}")

    # Save the adversarial images
    #for j, adv_image in enumerate(adv_images):
        # Convert the tensor to an image
    #    adv_image = transforms.ToPILImage()(adv_image)

        # Get the predicted label for the current image
    #    pred_label_name = get_label_name(preds[j].item())[1]

        # Save the image with the filename set to its classification along with its indices i and j
    #    adv_image.save(f'/content/test/adv_image_{pred_label_name}_{i}_{j}.png')

print(f"The attack took {round(total_attack_time, 2)} sec to complete.")

The attack took 48.38 sec to complete.


## **Calculate the robust accuracy and the confusion matrix** of the ResNet50 model based on its performance after feeding the resulting images of the adversarial attack.

In [289]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

print(f"Predicted labels: {predicted_labels}")
print(f"Actual labels: {true_labels}")

# Calculate the accuracy score
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy * 100}%')

# Calculate the precision score
precision = precision_score(true_labels, predicted_labels, average='macro')
print(f'Precision: {precision * 100}%')

# Calculate the recall score
recall = recall_score(true_labels, predicted_labels, average='macro')
print(f'Recall: {recall * 100}%')

# Calculate the F1 score
f1 = f1_score(true_labels, predicted_labels, average='macro')
print(f'F1 Score: {f1 * 100}%')

Predicted labels: ['Granny_Smith', 'pomegranate', 'Granny_Smith', 'hip', 'strawberry', 'bell_pepper', 'candle', 'Granny_Smith', 'Granny_Smith', 'pomegranate', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'buckeye', 'hip', 'hip', 'head_cabbage', 'buckeye', 'hip', 'fig', 'hip', 'pomegranate', 'plunger', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper']
Actual labels: ['Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'Granny_Smith', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'beetroot', 'beetroot', 'beetroot', 'beetroot', 'beetroot', 'beetroot', 'beetroot', 'beetroot', 'beetroot', 'beetroot', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_pepper', 'bell_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## The code below will be used to test the resulting images of the attacks.
This is to manually check if the resulting classifications of each image matches the correct computation of the robust accuracy, f1-scores and the confusion matrix.

In [290]:
import torch
from torchvision import models, transforms

**11. Load Pre-Trained Model:**

In [291]:
model = models.resnet50(pretrained=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

**12. Preprocess the Image:**

In [292]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

**13. Load and Preprocess Your Image:**

In [293]:
from PIL import Image

img = Image.open("/content/test/adv_image_banana_0_10.png").convert("RGB")
img_t = preprocess(img)
batch_t = torch.unsqueeze(img_t, 0)

**14. Download the ImageNet Class Index File:** You can download the ImageNet class index file (a JSON file that maps class indices to labels) from this link. Save it in your working directory.

**15. Load the Class Index File:** You can load the class index file using the json module in Python. The loaded object is a dictionary that maps indices to class labels.

In [294]:
import json

# Upload the json file to google colab first before running this.
with open("imagenet_class_index.json") as f:
    class_idx = json.load(f)

idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

In this code, idx2label is a list of class labels in the correct order.

**16. Get the class name of the predicted class:**

In [295]:
with torch.no_grad():
    output = model(batch_t)
_, predicted_idx = torch.max(output, 1)

# Get the name of the predicted class
predicted_class = idx2label[predicted_idx.item()]

print(f"The model predicts that the image belongs to: {predicted_class}.")

The model predicts that the image belongs to: banana.


**References**

https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition